### corona데이터 로드
1. 데이터 로드
2. [Unnamed: 0,seq,stateDt,statetime,updateDt] 컬럼을 삭제
3. 인덱스의 값을 createDt로 변경(시계열로 변경하여 인덱스로 이동)
4. 컬럼의 이름을 변경 : 사망자, 확진자, 누적의심자,누적확진율
5. 인덱스 이름 변경 : 등록일시
6. 일일확진자 ,일일사망자 컬럼 추가 사망자,확진자의 차이를 대입 결측치는 0으로 대체
7. 누적의심자, 누적확진율에 결측치 데이터는 전의 데이터 값으로 대체
8. 인덱스 기준으로 오름차순 정렬
9. 일일확진자가 음수인 값을 찾아서 인덱스를 삭제

In [87]:
#1
import pandas as pd

## 1,2,3번 한번에하기
# df = pd.read_csv("../221214/corona.csv",usecols=[1,2,3,8,9],parse_dates=[0],index_col="createDt")
df = pd.read_csv("../221214/corona.csv")
df

,Unnamed: 0,createDt,deathCnt,decideCnt,seq,stateDt,stateTime,updateDt,accExamCnt,accDefRate
0,0,2022-12-14 00:42:41.0,31174,27925572,1099,20221214,00:00,NaN,NaN,NaN
1,1,2022-12-13 04:34:20.0,31128,27841001,1098,20221213,00:00,NaN,NaN,NaN
2,2,2022-12-12 00:34:36.0,31099,27754149,1097,20221212,00:00,NaN,NaN,NaN
3,3,2022-12-11 01:10:06.0,31069,27728482,1096,20221211,00:00,NaN,NaN,NaN
4,4,2022-12-10 00:41:17.0,31029,27674163,1095,20221210,00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1054,1054,2020-01-24 00:00:00.000,0,2,5,20200124,00:00,2021-10-07 10:30:51.51,29.0,6.896552
1055,1055,2020-01-23 09:00:00.000,0,1,4,20200123,09:00,2021-10-07 10:30:51.51,25.0,4.000000
1056,1056,2020-01-22 12:00:00.000,0,1,3,20200122,12:00,2021-10-07 10:30:51.51,21.0,4.761905
1057,1057,2020-01-21 09:00:00.000,0,1,2,20200121,09:00,2021-10-07 10:30:51.51,15.0,6.666667


In [88]:
#2
df.drop(["Unnamed: 0",'seq','stateDt','stateTime','updateDt'],axis=1,inplace=True)

In [89]:
#3
df["createDt"] = pd.to_datetime(df["createDt"])
df.set_index("createDt",inplace=True)

In [90]:
#4
df.columns = ['사망자', '확진자', '누적의심자','누적확진율' ]

In [91]:
#5
df.index.name = "등록일시"

In [92]:
#6
df['일일사망자'] = df["사망자"].diff(-1).fillna(0)
df['일일확진자'] = df['확진자'].diff(-1).fillna(1)

In [93]:
#7
df["누적의심자"].fillna(method="bfill",inplace=True)
df["누적확진율"].fillna(method='bfill',inplace=True)

In [94]:
df.sort_index(inplace=True)

In [95]:
ind = df["일일확진자"]<0
df[ind]
df = df[~ind]

In [96]:
df

,사망자,확진자,누적의심자,누적확진율,일일사망자,일일확진자
등록일시,,,,,,
2020-01-20 17:00:00,0,1,4.0,25.000000,0.0,1.0
2020-01-21 09:00:00,0,1,15.0,6.666667,0.0,0.0
2020-01-22 12:00:00,0,1,21.0,4.761905,0.0,0.0
2020-01-23 09:00:00,0,1,25.0,4.000000,0.0,0.0
2020-01-24 00:00:00,0,2,29.0,6.896552,0.0,1.0
...,...,...,...,...,...,...
2022-12-10 00:41:17,31029,27674163,21518073.0,2.787908,54.0,62738.0
2022-12-11 01:10:06,31069,27728482,21518073.0,2.787908,40.0,54319.0
2022-12-12 00:34:36,31099,27754149,21518073.0,2.787908,30.0,25667.0
